In [ ]:
import IPython.display as ipd
import numpy as np
import pyaudio, wave
import math
import matplotlib.pyplot as plt
from numpy.matlib import repmat
from scipy.fftpack import dct
def Preemphasizing(chunk):
    pe=np.zeros(len(chunk))
    pe[0]=chunk[0]
    for i in range(1,len(chunk)):
        pe[i]=chunk[i]-0.95*chunk[i-1]
    return pe
def windowing(frame,mode="Hamming"):
    n=len(frame)
    if mode=="Hamming":
        frame=frame*np.hamming(n)
    elif mode=="Hanning":
        frame=frame*np.hanning(n)
    elif mode=="Blackman":
        frame=frame*np.blackman(n)
    return frame
def zero_padding(frame,FFT_size=512):
    #zero padding for FFT
    frame_length=len(frame)
    padding_length=FFT_size-frame_length
    zeros=np.zeros((padding_length,))
    zero_padding_frame=np.concatenate((frame,zeros))
    return zero_padding_frame
def FFT(frame,FFT_size=512):
    mag_frame = np.absolute(np.fft.rfft(frame, FFT_size))
    power_spectrum = ((1.0 / FFT_size) * ((mag_frame) ** 2))  
    return power_spectrum
def hztomel(f):
    z = 2595 * np.log10(1+f/700)
    return z
def meltohz(z):
    f = 700*(np.power(10,z/2595)-1)  
    return f
def filterbanks(power_spectrum,lower_boundHz=133.33,upper_boundHz=6855.4976,num_banks=40,FFT_size=512):
    upper_boundMel=hztomel(upper_boundHz)
    lower_boundMel=hztomel(lower_boundHz)
    mel_points=np.linspace(lower_boundMel,upper_boundMel,num_banks+2)
    hz_points=meltohz(mel_points)
    bin=np.floor((FFT_size + 1)/2*hz_points/(upper_boundHz-lower_boundHz))
    bin=bin-bin[0]
    fbank=np.zeros((num_banks,int(np.floor(FFT_size/2+1))))
    for m in range(1,num_banks+1):
        f_m_low=int(bin[m-1])   
        f_m_center=int(bin[m])  
        f_m_up=int(bin[m+1])  
        for k in range(f_m_low,f_m_center):
            fbank[m-1,k]=(k-bin[m-1])/(bin[m]-bin[m-1])
        for k in range(f_m_center,f_m_up):
            fbank[m-1,k]=(bin[m+1]-k)/(bin[m+1]-bin[m])
    Fb=np.dot(power_spectrum,fbank.T)
    Fb=np.where(Fb==0,np.finfo(float).eps,Fb) 
    Log_Mel_spectrum=10*np.log(Fb) 
    
    return Log_Mel_spectrum